In [5]:
import pandas as pd
import sys
import mlflow

sys.path.append("..")
from common.constants import DATAFOLDER, ASSET_INFO
from preprocess.src.extract_data import splitted_data_timestamps

In [6]:
def splitted_data_index(n_splits: int, asset_name: str, asset_id: int):
    data = pd.read_parquet(os.path.join("../data/", f"{asset_name}.parquet.gzip"), engine="pyarrow")
    
    _timestamps = data.index.values
    split_length = len(_timestamps) // n_splits

    splitted_timestamps = {}
    first_timestamps = 0
    for i in range(split_length, len(_timestamps), split_length):
        splitted_timestamps[i // split_length - 1] = _timestamps[first_timestamps:i]
        first_timestamps = i
    splitted_timestamps[len(_timestamps)//split_length - 1] = _timestamps[first_timestamps:]
    return {"asset_name": asset_name, "asset_id": asset_id, "splitted_indexes": splitted_timestamps}


res = splitted_data_index(n_splits=6, asset_name="Bitcoin", asset_id=1)
_timestamps = res["splitted_indexes"]

for key in _timestamps.keys():
    item = _timestamps[key]
    print(key, len(item))

0 326160
1 326160
2 326160
3 326160
4 326160
5 326160


In [92]:
import itertools
import numpy as np

folds = list(res["splitted_indexes"].keys())
n_folds = len(folds)
n_test_folds = 2

selected_fold_bounds = list(itertools.combinations(folds, n_test_folds))

n_splits = 6
num_senarios = int(len(selected_fold_bounds) / (n_splits/n_test_folds))

cpcv_indexes = []
for i in range(num_senarios):
    _jump_indexes = list(range(num_senarios-1, 0, -1))[:i]
    
    while len(_jump_indexes) < num_senarios - 1:
        _jump_indexes.append(1)

    test_folds_indexes = []
    test_folds_indexes.append(i)
    for jump_idx, jump_val in enumerate(_jump_indexes):
        test_folds_indexes.append(test_folds_indexes[jump_idx]+jump_val)
    
    _senario = {}
    _checked_valid_fold_indexes = []
    for idx, test_fold_index in enumerate(test_folds_indexes):
        _senario[idx] = {"train": [], "valid": []}

        for valid_fold_idx in selected_fold_bounds[test_fold_index]:
            if valid_fold_idx not in _checked_valid_fold_indexes:
                _senario[idx]["valid"].append(valid_fold_idx)
                _checked_valid_fold_indexes.append(valid_fold_idx)

        _senario[idx]["train"] += [v for v in range(n_folds) if v not in selected_fold_bounds[test_fold_index]]
    print(_senario)
    cpcv_indexes.append(_senario)
        


{0: {'train': [2, 3, 4, 5], 'valid': [0, 1]}, 1: {'train': [1, 3, 4, 5], 'valid': [2]}, 2: {'train': [1, 2, 4, 5], 'valid': [3]}, 3: {'train': [1, 2, 3, 5], 'valid': [4]}, 4: {'train': [1, 2, 3, 4], 'valid': [5]}}
{0: {'train': [1, 3, 4, 5], 'valid': [0, 2]}, 1: {'train': [0, 3, 4, 5], 'valid': [1]}, 2: {'train': [0, 2, 4, 5], 'valid': [3]}, 3: {'train': [0, 2, 3, 5], 'valid': [4]}, 4: {'train': [0, 2, 3, 4], 'valid': [5]}}
{0: {'train': [1, 2, 4, 5], 'valid': [0, 3]}, 1: {'train': [0, 2, 4, 5], 'valid': [1]}, 2: {'train': [0, 1, 4, 5], 'valid': [2]}, 3: {'train': [0, 1, 3, 5], 'valid': [4]}, 4: {'train': [0, 1, 3, 4], 'valid': [5]}}
{0: {'train': [1, 2, 3, 5], 'valid': [0, 4]}, 1: {'train': [0, 2, 3, 5], 'valid': [1]}, 2: {'train': [0, 1, 3, 5], 'valid': [2]}, 3: {'train': [0, 1, 2, 5], 'valid': [3]}, 4: {'train': [0, 1, 2, 4], 'valid': [5]}}
{0: {'train': [1, 2, 3, 4], 'valid': [0, 5]}, 1: {'train': [0, 2, 3, 4], 'valid': [1]}, 2: {'train': [0, 1, 3, 4], 'valid': [2]}, 3: {'train': [

In [104]:
for cpcv_index in cpcv_indexes:
    print("="*10)
    for key in cpcv_index.keys():
        o_x = ["?" for _ in range(n_folds)]
        for train_idx in cpcv_index[key]["train"]:
            o_x[train_idx] = "o"
        for valid_idx in cpcv_index[key]["valid"]:
            o_x[valid_idx] = "x"
        print(*o_x)

x x o o o o
? o x o o o
? o o x o o
? o o o x o
? o o o o x
x o x o o o
o x ? o o o
o ? o x o o
o ? o o x o
o ? o o o x
x o o x o o
o x o ? o o
o o x ? o o
o o ? o x o
o o ? o o x
x o o o x o
o x o o ? o
o o x o ? o
o o o x ? o
o o o ? o x
x o o o o x
o x o o o ?
o o x o o ?
o o o x o ?
o o o o x ?


In [11]:
print(_data_indexes - _timestamps)

[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
 -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200. -1200.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]


In [18]:
coin_data = coin_data.reindex(range(coin_data.index[0],coin_data.index[-1]+60,60),method='pad')

In [21]:
for i in range(end-60, end+60, 60):
    print(i)

1632182340
1632182400
